In [2]:
import numpy as np
from scipy.optimize import minimize

def isda_discount_factor(rates, time_points, mean_reversion, volatility):
    num_time_steps = len(time_points)
    discount_factors = np.zeros(num_time_steps)
    discount_factors[0]=1

    for t in range(1, num_time_steps):
        dt = time_points[t] - time_points[t-1]
        a = mean_reversion

        # Define the objective function for optimization
        objective_function = lambda x: (x * (1 - np.exp(-a * dt)) / a - np.exp(-x * dt) * discount_factors[t-1]) ** 2

        # Use optimization to find the optimal rate for each tenor
        result = minimize(objective_function, rates[t-1], bounds=[(0, 1)])

        # Update discount factor using the optimal rate
        discount_factors[t] = np.exp(-result.x * dt) * discount_factors[t-1]

    return discount_factors

def isda_hazard_rate(cds_spread, discount_factors, recovery_rate, notional, maturity):
    # Calculate the implied hazard rate using the CDS spread and discount factors
    protection_leg_pv = notional * cds_spread * (1 - recovery_rate) * (1 - discount_factors[-1])
    premium_leg_pv = notional * (1 - recovery_rate) * discount_factors[-1]

    hazard_rate = -np.log(1 - protection_leg_pv / premium_leg_pv) / maturity

    return hazard_rate

def isda_cds_pricing(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility):
    # Perform curve bootstrapping to get discount factors
    discount_factors = isda_discount_factor(market_rates, time_points, mean_reversion, volatility)

    # Calculate the implied hazard rate using CDS spread and discount factors
    hazard_rate = isda_hazard_rate(cds_spread, discount_factors, recovery_rate, notional, maturity)

    # Calculate present value of protection leg (premiums)
    protection_leg_pv = notional * cds_spread * (1 - recovery_rate) * (1 - discount_factors[-1])

    # Calculate present value of premium leg (default payments)
    premium_leg_pv = notional * (1 - recovery_rate) * discount_factors[-1]

    # Calculate CDS price
    cds_price = protection_leg_pv - premium_leg_pv

    return cds_price #, discount_factors, hazard_rate

def isda_cds_npv(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility):
    # Perform curve bootstrapping to get discount factors
    discount_factors = isda_discount_factor(market_rates, time_points, mean_reversion, volatility)

    # Calculate the implied hazard rate using CDS spread and discount factors
    hazard_rate = isda_hazard_rate(cds_spread, discount_factors, recovery_rate, notional, maturity)

    # Calculate present value of protection leg (premiums)
    protection_leg_pv = notional * cds_spread * (1 - recovery_rate) * np.sum((1 - discount_factors[:-1]))

    # Calculate present value of premium leg (default payments)
    premium_leg_pv = notional * (1 - recovery_rate) * discount_factors[-1]

    # Calculate CDS NPV
    cds_npv = protection_leg_pv - premium_leg_pv

    return cds_npv

def cds_npv_sensitivity_analysis(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility, perturbation):
    # Calculate the base CDS NPV
    base_cds_npv = isda_cds_npv(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility)

    # Perturb the market rates
    perturbed_market_rates = market_rates + perturbation

    # Calculate the perturbed CDS NPV
    perturbed_cds_npv = isda_cds_npv(notional, cds_spread, maturity, recovery_rate, perturbed_market_rates, time_points, mean_reversion, volatility)

    # Calculate the CDS NPV delta (sensitivity)
    cds_npv_delta = (perturbed_cds_npv - base_cds_npv) / perturbation

    return cds_npv_delta 

def cds_curve_shift_in_parallel(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility, shift):
    # Perform a parallel shift in the CDS curve
    shifted_market_rates = market_rates + shift

    # Calculate CDS NPV with the shifted curve
    cds_curve_npv_shifted = isda_cds_npv(notional, cds_spread, maturity, recovery_rate, shifted_market_rates, time_points, mean_reversion, volatility)

    return cds_curve_npv_shifted

def cds_spread_curve_shift_in_parallel(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility, spread_shift):
    # Perform a parallel shift in the CDS spread curve
    shifted_cds_spread = cds_spread + spread_shift

    # Calculate CDS NPV with the shifted spread curve
    cds_spread_npv_shifted = isda_cds_npv(notional, shifted_cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility)

    return cds_spread_npv_shifted

def cds_amount_to_sell_delta(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility, shift_delta):
    # Calculate CDS NPV with the original curve
    cds_npv_base = isda_cds_npv(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility)

    # Calculate CDS NPV with a slightly shifted curve
    cds_npv_shifted = cds_spread_curve_shift_in_parallel(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility, shift)

    # Calculate the finite difference approximation of the delta
    delta = (cds_npv_shifted - cds_npv_base) / shift

    return delta

def isda_swap_fixed_leg(notional, fixed_rate, fixed_leg_tenor, maturity, market_rates, time_points, mean_reversion, volatility):
    discount_factors = isda_discount_factor(market_rates, time_points, mean_reversion, volatility)

    fixed_leg_cash_flows = np.zeros_like(time_points)
    fixed_leg_cash_flows[::fixed_leg_tenor] = fixed_rate * notional * fixed_leg_tenor

    # Discount fixed leg cash flows
    present_value_fixed_leg = np.sum(fixed_leg_cash_flows * discount_factors)

    return present_value_fixed_leg

def isda_swap_float_leg(notional, floating_leg_tenor, maturity, market_rates, time_points, mean_reversion, volatility):
    discount_factors = isda_discount_factor(market_rates, time_points, mean_reversion, volatility)

    floating_leg_cash_flows = np.zeros_like(time_points)
    floating_leg_cash_flows[::floating_leg_tenor] = notional * floating_leg_tenor

    # Discount floating leg cash flows
    present_value_floating_leg = np.sum(floating_leg_cash_flows * discount_factors)

    return present_value_floating_leg

def isda_cds_premium_leg(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility):
    # Perform curve bootstrapping to get discount factors
    discount_factors = isda_discount_factor(market_rates, time_points, mean_reversion, volatility)

    # Calculate the implied hazard rate using CDS spread and discount factors
    hazard_rate = isda_hazard_rate(cds_spread, discount_factors, recovery_rate, notional, maturity)

    # Calculate present value of premium leg (default payments)
    premium_leg_cash_flows = notional * (1 - recovery_rate) * discount_factors[:-1]

    # Discount premium leg cash flows
    present_value_premium_leg = np.sum(premium_leg_cash_flows)

    return present_value_premium_leg

def isda_cds_default_leg(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility):
    discount_factors = isda_discount_factor(market_rates, time_points, mean_reversion, volatility)

    # Calculate the present value of default leg cash flows
    default_leg_cash_flows = notional * (1 - recovery_rate) * (discount_factors[:-1] - discount_factors[1:])

    # Discount default leg cash flows
    present_value_default_leg = np.sum(default_leg_cash_flows)

    return present_value_default_leg

# Example parameters
notional = 1  # Notional amount
cds_spread = 0.02   # CDS spread (annualized)
maturity = 5         # Maturity in years
recovery_rate = 0.4  # Recovery rate in case of default
mean_reversion = 0.1
volatility = 0.01
shift = 0.005       # Parallel shift in the CDS curve (adjust as needed)
spread_shift = 0.005  # Parallel shift in the CDS spread curve (adjust as needed)
perturbation=0.0001
shift_delta=0.001
business_days=252

fixed_rate = 0.03   # Fixed interest rate
floating_leg_tenor = 1  # Floating leg tenor in years
fixed_leg_tenor = 1     # Fixed leg tenor in years

floating_leg_tenor = 1  # Floating leg tenor in years
fixed_leg_tenor = 1     # Fixed leg tenor in years
# Example market rates for curve bootstrapping
yearly_rates = np.array([0.02, 0.025, 0.03, 0.035, 0.04])
market_rates =[]
for i in range(len(yearly_rates)):
    market_rates=market_rates+[yearly_rates[i]]*business_days
    
market_rates =np.array(market_rates)

# Time parameters
num_time_steps = business_days * maturity  # assuming daily time steps
time_points = np.linspace(0, maturity, num_time_steps + 1)

# Perform CDS price sensitivity analysis
base_cds_price = isda_cds_pricing(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility)
# Perform CDS NPV and NPV senssitivity analysis
cds_npv_base = isda_cds_npv(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility)
cds_curve_npv_shifted = cds_curve_shift_in_parallel(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility, shift)
cds_spread_npv_shifted = cds_spread_curve_shift_in_parallel(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility, spread_shift)

# Calculate the change in NPV
change_in_npv = cds_spread_npv_shifted - cds_npv_base

# Calculate the delta of the CDS amount to sell
cds_amount_delta = cds_amount_to_sell_delta(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility,shift_delta)

# calculate premium and default legs
premium_leg =isda_cds_premium_leg(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility)
default_leg =isda_cds_default_leg(notional, cds_spread, maturity, recovery_rate, market_rates, time_points, mean_reversion, volatility)

#
float_leg=isda_swap_float_leg(notional, floating_leg_tenor, maturity, market_rates, time_points, mean_reversion, volatility)
fixed_leg =isda_swap_fixed_leg(notional, fixed_rate, fixed_leg_tenor, maturity, market_rates, time_points, mean_reversion, volatility)

# Print the results
print(f"Base CDS Price: {base_cds_price:.6f}")
print(f"Base CDS NPV: {cds_npv_base:.6f}")
print(f"CDS NPV with {shift:.3%} parallel shift in the curve: {cds_curve_npv_shifted:.6f}")
print(f"CDS NPV with {spread_shift:.3%} parallel shift in the spread curve: {cds_spread_npv_shifted:.6f}")
print(f"CDS in NPV with {spread_shift:.3%} parallel shift in the spread curve:{change_in_npv:.6f} ")
print(f"Delta of CDS amount to sell: {cds_amount_delta:.6f}")
print(f"CDS Float Leg: {float_leg:.6f}")
print(f"CDS Fixed Leg: {fixed_leg:.6f}")
print(f"CDS Premium Leg: {premium_leg:.6f}")
print(f"CDS Default Leg: {default_leg:.6f}")



C:\Users\a.mernissi\AppData\Local\Temp\ipykernel_65152\3895938002.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  discount_factors[t] = np.exp(-result.x * dt) * discount_factors[t-1]
C:\Users\a.mernissi\AppData\Local\Temp\ipykernel_65152\3895938002.py:29: RuntimeWarning: invalid value encountered in log
  hazard_rate = -np.log(1 - protection_leg_pv / premium_leg_pv) / maturity


Base CDS Price: 0.007876
Base CDS NPV: 12.106369
CDS NPV with 0.500% parallel shift in the curve: 12.106369
CDS NPV with 0.500% parallel shift in the spread curve: 15.133972
CDS in NPV with 0.500% parallel shift in the spread curve:3.027603 
Delta of CDS amount to sell: 605.520602
CDS Float Leg: 250.805735
CDS Fixed Leg: 7.524172
CDS Premium Leg: 150.479398
CDS Default Leg: 0.595957


In [4]:
import numpy as np
from scipy.optimize import minimize

def isda_discount_factor(effective_date, execution_date, yield_curve, time_points, mean_reversion, volatility, day_count_convention, payment_frequency):
    num_time_steps = len(time_points)
    discount_factors = np.zeros(num_time_steps)

    for t in range(1, num_time_steps):
        dt = time_points[t] - time_points[t-1]
        a = mean_reversion

        # Adjust effective date and execution date
        t_effective = effective_date - time_points[t-1]
        t_exec = execution_date - time_points[t-1]

        # Adjust dt based on day count convention and payment frequency
        dt_adjusted = dt / payment_frequency

        if day_count_convention == 'ACT/360':
            dt_adjusted = dt_adjusted / 360
        elif day_count_convention == 'ACT/365':
            dt_adjusted = dt_adjusted / 365
        # Add other day count conventions as needed

        # Define the objective function for optimization
        objective_function = lambda x: (x * (1 - np.exp(-a * t_effective)) / a - np.exp(-x * t_effective) * discount_factors[t-1]) ** 2

        # Use optimization to find the optimal rate for each tenor
        result = minimize(objective_function, yield_curve[t-1], bounds=[(0, 1)])

        # Update discount factor using the optimal rate
        discount_factors[t] = np.exp(-result.x * t_exec) * discount_factors[t-1]

    return discount_factors

def isda_hazard_rate(effective_date, execution_date, credit_event_date, cds_spread, discount_factors, recovery_rate, notional, maturity, day_count_convention, payment_frequency, seniority_level, reference_entity_level):
    # Adjust effective date, execution date, and credit event date
    t_effective = effective_date - discount_factors[0]
    t_exec = execution_date - discount_factors[0]
    t_credit_event = credit_event_date - discount_factors[0]

    # Adjust time parameters based on day count convention and payment frequency
    t_exec = t_exec / payment_frequency
    t_credit_event = t_credit_event / payment_frequency

    if day_count_convention == 'ACT/360':
        t_exec = t_exec / 360
        t_credit_event = t_credit_event / 360
    elif day_count_convention == 'ACT/365':
        t_exec = t_exec / 365
        t_credit_event = t_credit_event / 365
    # Add other day count conventions as needed

    # Adjust recovery rate based on seniority level and reference entity level
    if seniority_level == 'senior' and reference_entity_level == 'entity_A':
        recovery_rate_adjusted = recovery_rate_senior_entity_A
    elif seniority_level == 'senior' and reference_entity_level == 'entity_B':
        recovery_rate_adjusted = recovery_rate_senior_entity_B
    # Add other combinations of seniority level and reference entity level as needed

    # Calculate the implied hazard rate using the CDS spread and discount factors
    protection_leg_pv = notional * cds_spread * (1 - recovery_rate_adjusted) * np.sum((1 - discount_factors[:t_credit_event]))
    premium_leg_pv = notional * (1 - recovery_rate_adjusted) * discount_factors[t_credit_event]

    hazard_rate = -np.log(1 - protection_leg_pv / premium_leg_pv) / t_exec

    return hazard_rate

def isda_cds_npv(effective_date, execution_date, credit_event_date, notional, cds_spread, maturity, recovery_rate, yield_curve, time_points, mean_reversion, volatility, day_count_convention, payment_frequency, seniority_level, reference_entity_level):
    # Perform curve bootstrapping to get discount factors
    discount_factors = isda_discount_factor(effective_date, execution_date, yield_curve, time_points, mean_reversion, volatility, day_count_convention, payment_frequency)

    # Calculate the implied hazard rate using CDS spread and discount factors
    hazard_rate = isda_hazard_rate(effective_date, execution_date, credit_event_date, cds_spread, discount_factors, recovery_rate, notional, maturity, day_count_convention, payment_frequency, seniority_level, reference_entity_level)

    # Calculate present value of protection leg (premiums)
    protection_leg_pv = notional * cds_spread * (1 - recovery_rate) * np.sum((1 - discount_factors[:credit_event_date]))

    # Calculate present value of premium leg (default payments)
    premium_leg_pv = notional * (1 - recovery_rate) * discount_factors[credit_event_date]

    # Calculate CDS NPV
    cds_npv = protection_leg_pv - premium_leg_pv

    return cds_npv



# Example parameters
effective_date = 1.0        # Effective date in years
execution_date = 2.0        # Execution date in years
credit_event_date = 4.0     # Credit event date in years
notional = 1000000           # Notional amount
cds_spread = 0.02            # CDS spread (annualized)
maturity = 5                  # Maturity in years
recovery_rate_senior_entity_A = 0.4    # Recovery rate for senior debt of entity A in case of default
recovery_rate_senior_entity_B = 0.3    # Recovery rate for senior debt of entity B in case of default
mean_reversion = 0.1
volatility = 0.01
day_count_convention = 'ACT/360'  # Day count convention
payment_frequency = 4         # Payment frequency (e.g., quarterly)
seniority_level = 'senior'    # Seniority level ('senior' or 'subordinated')
reference_entity_level = 'entity_A'  # Reference entity level ('entity_A' or 'entity_B')

# Example yield curve for curve bootstrapping
yield_curve = np.array([0.02, 0.025, 0.03, 0.035, 0.04])

# Calculate the CDS NPV
cds_npv = isda_cds_npv(effective_date, execution_date, credit_event_date, notional, cds_spread, maturity, recovery_rate_senior_entity_A, yield_curve, time_points, mean_reversion, volatility, day_count_convention, payment_frequency, seniority_level, reference_entity_level)

# Print the result
print(f"CDS NPV: {cds_npv:.6f}")


TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
import numpy as np
from scipy.optimize import minimize

def isda_discount_factor(effective_date, execution_date, yield_curve, time_points, mean_reversion, volatility, day_count_convention, payment_frequency):
    num_time_steps = len(time_points)
    discount_factors = np.zeros(num_time_steps)

    for t in range(1, num_time_steps):
        dt = time_points[t] - time_points[t-1]
        a = mean_reversion

        # Adjust effective date and execution date
        t_effective = effective_date - time_points[t-1]
        t_exec = execution_date - time_points[t-1]

        # Adjust dt based on day count convention and payment frequency
        dt_adjusted = dt / payment_frequency

        if day_count_convention == 'ACT/360':
            dt_adjusted = dt_adjusted / 360
        elif day_count_convention == 'ACT/365':
            dt_adjusted = dt_adjusted / 365
        # Add other day count conventions as needed

        # Define the objective function for optimization
        objective_function = lambda x: (x * (1 - np.exp(-a * t_effective)) / a - np.exp(-x * t_effective) * discount_factors[t-1]) ** 2

        # Use optimization to find the optimal rate for each tenor
        result = minimize(objective_function, yield_curve[t-1], bounds=[(0, 1)])

        # Update discount factor using the optimal rate
        discount_factors[t] = np.exp(-result.x * t_exec) * discount_factors[t-1]

    return discount_factors

def isda_hazard_rate(effective_date, execution_date, credit_event_date, cds_spread, discount_factors, recovery_rate, notional, maturity, day_count_convention, payment_frequency, seniority_level, reference_entity_level):
    # Adjust effective date, execution date, and credit event date
    t_effective = effective_date - discount_factors[0]
    t_exec = execution_date - discount_factors[0]
    t_credit_event = credit_event_date - discount_factors[0]

    # Adjust time parameters based on day count convention and payment frequency
    t_exec = t_exec / payment_frequency
    t_credit_event = t_credit_event / payment_frequency

    if day_count_convention == 'ACT/360':
        t_exec = t_exec / 360
        t_credit_event = t_credit_event / 360
    elif day_count_convention == 'ACT/365':
        t_exec = t_exec / 365
        t_credit_event = t_credit_event / 365
    # Add other day count conventions as needed

    # Adjust recovery rate based on seniority level and reference entity level
    if seniority_level == 'senior' and reference_entity_level == 'entity_A':
        recovery_rate_adjusted = recovery_rate_senior_entity_A
    elif seniority_level == 'senior' and reference_entity_level == 'entity_B':
        recovery_rate_adjusted = recovery_rate_senior_entity_B
    # Add other combinations of seniority level and reference entity level as needed

    # Calculate the implied hazard rate using the CDS spread and discount factors
    protection_leg_pv = notional * cds_spread * (1 - recovery_rate_adjusted) * np.sum((1 - discount_factors[:t_credit_event]))
    premium_leg_pv = notional * (1 - recovery_rate_adjusted) * discount_factors[t_credit_event]

    hazard_rate = -np.log(1 - protection_leg_pv / premium_leg_pv) / t_exec

    return hazard_rate

def isda_cds_npv(effective_date, execution_date, credit_event_date, notional, cds_spread, maturity, recovery_rate, yield_curve, time_points, mean_reversion, volatility, day_count_convention, payment_frequency, seniority_level, reference_entity_level):
    # Perform curve bootstrapping to get discount factors
    discount_factors = isda_discount_factor(effective_date, execution_date, yield_curve, time_points, mean_reversion, volatility, day_count_convention, payment_frequency)

    # Calculate the implied hazard rate using CDS spread and discount factors
    hazard_rate = isda_hazard_rate(effective_date, execution_date, credit_event_date, cds_spread, discount_factors, recovery_rate, notional, maturity, day_count_convention, payment_frequency, seniority_level, reference_entity_level)

    # Calculate present value of protection leg (premiums)
    protection_leg_pv = notional * cds_spread * (1 - recovery_rate) * np.sum((1 - discount_factors[:credit_event_date]))

    # Calculate present value of premium leg (default payments)
    premium_leg_pv = notional * (1 - recovery_rate) * discount_factors[credit_event_date]

    # Calculate CDS NPV
    cds_npv = protection_leg_pv - premium_leg_pv

    return cds_npv

# Example parameters
effective_date = 1.0        # Effective date in years
execution_date = 2.0        # Execution date in years
credit_event_date = 4.0     # Credit event date in years
notional = 1000000           # Notional amount
cds_spread = 0.02            # CDS spread (annualized)
maturity = 5                  # Maturity in years
recovery_rate_senior_entity_A = 0.4    # Recovery rate for senior debt of entity A in case of default
recovery_rate_senior_entity_B = 0.3    # Recovery rate for senior debt of entity B in case of default
mean_reversion = 0.1
volatility = 0.01
day_count_convention = 'ACT/360'  # Day count convention
payment_frequency = 4         # Payment frequency (e.g., quarterly)
seniority_level = 'senior'    # Seniority level ('senior' or 'subordinated')
reference_entity_level = 'entity_A'  # Reference entity level ('entity_A' or 'entity_B')

# Example yield curve for curve bootstrapping
yield_curve = np.array([0.02, 0.025, 0.03, 0.035, 0.04])

# Calculate the CDS NPV
cds_npv = isda_cds_npv(effective_date, execution_date, credit_event_date, notional, cds_spread, maturity, recovery_rate_senior_entity_A, yield_curve, time_points, mean_reversion, volatility, day_count_convention, payment_frequency, seniority_level, reference_entity_level)

# Print the result
print(f"CDS NPV: {cds_npv:.6f}")
